In [ ]:
# google_play_scraper 라이브러리 설치: 구글 플레이스토어에서 데이터를 쉽게 가져올 수 있게 해줌
!pip install google_play_scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.3 MB/s eta 0:00:00


In [ ]:
# 필요한 패키지와 모듈을 임포트
from google_play_scraper import Sort, reviews_all
import pandas as pd

# 'lang'과 'cont' 변수는 각각 리뷰의 언어와 국가를 설정합니다.
lang = 'ko'  # 리뷰의 언어를 한국어로 설정
cont = 'kr'  # 리뷰를 가져올 국가를 한국으로 설정

# reviews_all 함수를 사용해 앱의 모든 리뷰를 가져옵니다.
result = reviews_all(
    'com.selabs.speak',  # 앱의 패키지 이름
    sleep_milliseconds=3000,  # 리뷰 요청 사이의 대기 시간 설정
    lang=lang,  # 리뷰 언어 설정
    country=cont,  # 국가 설정
    sort=Sort.NEWEST  # 리뷰 정렬 방식 설정 (가장 관련성이 높은 순서)
)

# 수집된 리뷰 데이터를 저장할 리스트 초기화
reviews_data = []

# 수집된 모든 리뷰를 리스트에 추가
for item in result:
    review = {
        'reviewId': item['reviewId'],
        'content': item['content'],
        'score': item['score'],
        'thumbsUpCount': item['thumbsUpCount'],
        'reviewCreatedVersion': item['reviewCreatedVersion'],
        'at': item['at'],
        'replyContent': item.get('replyContent', None),
        'repliedAt': item.get('repliedAt', None),
        'appVersion': item.get('appVersion', None)
    }
    reviews_data.append(review)

# 리스트를 pandas DataFrame으로 변환
df_reviews = pd.DataFrame(reviews_data)
print(f'총 리뷰 개수: {len(df_reviews)}')  # 수집된 리뷰의 개수 출력


# # 리뷰 데이터를 CSV 파일로 저장
# csv_filename = 'reviews_speak.csv'
# df_reviews.to_csv(csv_filename, index=False, encoding='utf-8-sig', escapechar='\\')
# print(f'Reviews saved to {csv_filename}')

총 리뷰 개수: 15041


In [ ]:
from datetime import datetime

# 'at' 컬럼을 datetime 형식으로 변환
df_reviews['at'] = pd.to_datetime(df_reviews['at'])

# 특정 기간 필터링
start_date = '2020-01-01 00:00:00'
end_date = '2024-08-25 23:59:59'

filtered_reviews = df_reviews[(df_reviews['at'] >= start_date) & (df_reviews['at'] <= end_date)]

# 필터링된 리뷰 개수 출력
print(f'필터링된 리뷰 개수: {len(filtered_reviews)}')

필터링된 리뷰 개수: 14311


In [ ]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# Google Drive에 파일 저장
filtered_reviews.to_csv('/content/drive/MyDrive/speak_filtered_reviews.csv', encoding='utf-8-sig', index=False, escapechar='\\')

print("파일이 Google Drive에 저장되었습니다.")

Mounted at /content/drive
파일이 Google Drive에 저장되었습니다.


In [ ]:
import re
import pandas as pd

# 한국어 문자가 포함되었는지 확인하는 함수
def contains_korean(text):
    # 정규 표현식을 사용하여 한국어 문자가 포함되어 있는지 확인
    return bool(re.search(r'[ㄱ-ㅎㅏ-ㅣ가-힣]', text))

# 한국어 리뷰와 삭제된 리뷰를 저장할 리스트
korean_reviews_data = []
deleted_reviews_data = []

for item in result:
    content = item.get("content", "")

    # content가 문자열이 아니거나, 한국어가 포함되지 않은 경우
    if not isinstance(content, str) or not contains_korean(content):
        deleted_review = {
            "reviewId": item.get("reviewId"),
            "content": content,
            "score": item.get("score"),
            "thumbsUpCount": item.get("thumbsUpCount"),
            "reviewCreatedVersion": item.get("reviewCreatedVersion"),
            "at": item.get("at"),
            "replyContent": item.get("replyContent"),
            "repliedAt": item.get("repliedAt"),
            "appVersion": item.get("appVersion")
        }
        deleted_reviews_data.append(deleted_review)
    else:
        review = {
            "reviewId": item.get("reviewId"),
            "content": content,
            "score": item.get("score"),
            "thumbsUpCount": item.get("thumbsUpCount"),
            "reviewCreatedVersion": item.get("reviewCreatedVersion"),
            "at": item.get("at"),
            "replyContent": item.get("replyContent"),
            "repliedAt": item.get("repliedAt"),
            "appVersion": item.get("appVersion")
        }
        korean_reviews_data.append(review)

# 리스트를 DataFrame으로 변환
df_korean_reviews = pd.DataFrame(korean_reviews_data)
df_deleted_reviews = pd.DataFrame(deleted_reviews_data)

# 삭제된 리뷰 데이터프레임 출력
df_deleted_reviews

,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,0776ea12-8a98-41d2-81ae-003207c04ac8,Good,5,0,3.38.1,2024-08-25 01:29:30,None,NaT,3.38.1
1,6636d42b-e4ba-4a06-ae1b-6484df34d2a6,Good,5,0,3.38.1,2024-08-24 08:01:58,Thank you for your review.\n\nWe truly value y...,2024-08-26 08:30:23,3.38.1
2,51ddca8e-e159-4765-aa1b-d44d0f56ca5d,It's interesting. Try it.,5,0,3.39.0,2024-08-23 04:01:30,Thank you for your review.\n\nWe truly value y...,2024-08-23 06:23:52,3.39.0
3,bf1b45fb-3907-47c0-adcc-af0015c42205,Helpful Why don't you update over 40levels bad...,3,0,3.37.0,2024-08-17 20:31:13,"Hello, this is the Speak Support Team.\n\nWe'r...",2024-08-20 11:06:30,3.37.0
4,2ce3646a-4901-4a00-8052-b8a566a88745,It is very good app. Wonderful!,5,0,3.38.0,2024-08-17 07:50:49,Thank you for your review.\n\nWe truly value y...,2024-08-20 11:01:28,3.38.0
...,...,...,...,...,...,...,...,...,...
533,b4a2e8a0-50c4-4df6-b65d-7be8e5c2a8af,Good!!!,4,0,1.0.12,2019-05-21 12:41:45,"Thank you! We'll strive to be ""better!"" and ev...",2019-05-21 22:41:38,1.0.12
534,61ef411e-9c55-496a-afd5-bfd65caf2bae,good job,5,0,1.0.12,2019-05-20 12:16:03,Thank you. We'll strive to do a 'better' job a...,2019-05-20 23:37:04,1.0.12
535,d0dffca3-b7f6-43cf-a49c-d3bc391d6700,good!,5,0,1.0.12,2019-05-19 03:05:34,Thank you!,2019-05-20 02:33:06,1.0.12
536,9d005c07-0b63-4d5e-b27f-924d0515ef46,it's good,5,0,1.0.12,2019-05-17 00:00:47,"안녕하세요, Ga eun Fa 님.\n\ngood이 아니라 best가 될 때까지 발...",2019-05-18 01:27:23,1.0.12


In [ ]:
df_korean_reviews

,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,d0cd6889-b112-4851-9202-ead5ca654354,영어를재밌게혼자배울수있어요,5,0,3.38.1,2024-08-25 10:25:13,None,NaT,3.38.1
1,6eac1988-e65f-457a-871b-7f3b88a25217,a the at in on 이런 관사나 전치사의 쓰임이 이해가 너무 잘되요 충분히 ...,5,0,3.38.1,2024-08-25 03:09:43,None,NaT,3.38.1
2,8ccd4f38-93af-457a-b1ff-099eb1a447f7,반복에 반복 너무 좋습니다. 계속 말을 시켜서 상당히 좋습니다,5,0,3.39.0,2024-08-25 01:19:30,None,NaT,3.39.0
3,159d7876-74a4-4b3c-9607-e6da60ef7b2c,6일째 재미있게 하고 있어요,5,0,3.38.1,2024-08-25 00:11:15,None,NaT,3.38.1
4,44a760b3-8bc6-4d4b-bec6-f4d1e77cdf59,아 예기친 못한 오류 뭔데요; 프리톡 시작을 못하네 하 뭔 캐시삭제 재부팅 재설치 ...,1,0,3.39.0,2024-08-24 13:06:42,"안녕하세요, 스픽 서포트팀입니다.\n\n먼저 이용에 불편을 드려 정말 죄송합니다.\...",2024-08-26 08:34:45,3.39.0
...,...,...,...,...,...,...,...,...,...
14498,946e4020-5f49-46bc-bf61-5c37fe639583,탈퇴는 어떻게 해야되나요,1,0,1.0.7,2019-03-24 12:01:10,"안녕하세요, 신효주 님.\n\nfeedback@usespeak.com으로 가입한 계...",2019-03-25 19:28:24,1.0.7
14499,25cde7da-f994-4a12-8585-6559ba850c17,미리보기끝났네요. 괜찮은거같은데 결제할까말까고민중,5,0,1.0.7,2019-03-23 14:10:19,"안녕하세요, Marian Seo 님.\n\n오픈베타에 참여해 주셔서 감사합니다. 저...",2019-03-23 19:38:45,1.0.7
14500,25666f87-a0b2-405d-83e4-f1aefae0e667,좋아요,5,0,1.0.4,2019-03-16 11:43:25,안녕하세요. 저희 앱을 좋게 봐주셔서 감사합니다. 아직 시작에 불가하다고 생각합니다...,2019-03-23 04:36:43,1.0.4
14501,5ce6ecc2-1e6a-496d-9aa5-859e2a3efe05,강의와 스피킹 연습이 연계되어서 좋아요~,5,1,1.0.3,2019-03-15 08:38:32,안녕하세요 최진선 님.\n\n저희 앱이 진행되는 방식을 좋게 봐주셔서 감사합니다. ...,2019-03-23 04:36:13,1.0.3


In [ ]:
# Google Drive에 파일 저장
df_korean_reviews.to_csv('/content/drive/MyDrive/speak_korean_reviews.csv', encoding='utf-8-sig', index=False, escapechar='\\')
df_deleted_reviews.to_csv('/content/drive/MyDrive/speak_deleted_reviews.csv', encoding='utf-8-sig', index=False, escapechar='\\')

print("파일이 Google Drive에 저장되었습니다.")

파일이 Google Drive에 저장되었습니다.
